In [75]:
%load_ext autoreload
%autoreload 2
from EPECinterface import *
from gurobipy import *
import itertools
import csv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sets

In [76]:
dirty = ["coal", "gas"]
green = ["wind", "solar"]

countries = ["c1", "c2"]
producers = ["p1_1", "p1_2", "p2_1", "p2_2"]

domesticity = tuplelist([("c1","p1_1"), ("c1","p1_2"),
               ("c2","p2_1"), ("c2","p2_2")
              ])

scenario = ["s"+str(i+1) for i in range(2
                                       )]

data = inputData(dirty, green, countries, producers, domesticity, scenario)

In [77]:
mylist = readInputFromFile("inputs.csv")
baseData = mylist[0]
data = inputData(dirty, green, countries, producers, domesticity, scenario)
data.buildFromList(baseData)

In [78]:
data.cciInt = 200
data.cciSlope = 2

In [79]:
data

INSTANCE OF EPECinterface.inputData
***********************************

dirty ---------- ['coal', 'gas']

green ---------- ['wind', 'solar']

energy ---------- ['coal', 'gas', 'wind', 'solar']

countries ---------- ['c1', 'c2']

producers ---------- ['p1_1', 'p1_2', 'p2_1', 'p2_2']

domesticity ---------- <gurobi.tuplelist (4 tuples, 2 values each):
 ( c1 , p1_1 )
 ( c1 , p1_2 )
 ( c2 , p2_1 )
 ( c2 , p2_2 )
>

scenario ---------- ['s1', 's2']

LinInvCost ---------- {('p1_1', 'wind'): 15.0, ('p1_1', 'solar'): 25.0, ('p1_2', 'wind'): 15.0, ('p1_2', 'solar'): 25.0, ('p2_1', 'wind'): 15.0, ('p2_1', 'solar'): 25.0, ('p2_2', 'wind'): 15.0, ('p2_2', 'solar'): 25.0}

QuadInvCost ---------- {('p1_1', 'wind'): 1.0, ('p1_1', 'solar'): 4.0, ('p1_2', 'wind'): 1.0, ('p1_2', 'solar'): 4.0, ('p2_1', 'wind'): 1.0, ('p2_1', 'solar'): 4.0, ('p2_2', 'wind'): 1.0, ('p2_2', 'solar'): 4.0}

LinProdCost ---------- {('p1_1', 'coal'): 6.0, ('p1_1', 'gas'): 6.0, ('p1_1', 'wind'): 0.1, ('p1_1', 'solar'): 0.1, (

# Variables

In [64]:
M = Model()
# ## Variables
# ### Follower Variables 
PRODUCTION = M.addVars(data.producers, data.energy, data.scenario, 
                       vtype=GRB.CONTINUOUS, name = "PROD")
INVESTMENT = M.addVars(data.producers, data.energy, 
                       vtype=GRB.CONTINUOUS, name = "INV") 
FOLL_CARB_BUY = M.addVars(data.producers, lb = -GRB.INFINITY, 
                              name = "FOLL_CARB_BUY")

### All data.dirty investments must be 0 
for ee in data.dirty:
    for x in INVESTMENT.select("*",ee):
        x.ub = 0


# ### Leader Variables 
CARBON_PRICE = M.addVars(data.countries, name="CARB_PRI")
C_PROD = M.addVars(data.countries, data.scenario, name="C_PROD")
CARB_IMP = M.addVars(data.countries, lb=-GRB.INFINITY, name = "CARB_IMP")

TOTAL_INV = M.addVars(data.countries, data.energy, name = "TOTAL_INV")
TOTAL_EMIT = M.addVars(data.countries, data.scenario, name = "TOTAL_EMIT")
LEAD_CARB_BUY = M.addVars(data.countries, lb = -GRB.INFINITY, name = "LEAD_CARB_BUY") 

COUNTRY_OBJ = M.addVars(data.countries, lb= -10000+0*GRB.INFINITY, name = "COUNTRY_OBJ") 

# ### Duals 
D_INFRALIMIT = M.addVars(data.producers, data.energy, data.scenario,
                        name = "D_INFRA")
D_EMITLIMIT = M.addVars(data.producers, data.scenario, name = "D_EMIT")

# ### Binary variables 
B_PRODUCTION= M.addVars(data.producers, data.energy, data.scenario, 
                       vtype= GRB.BINARY, name = "B_Prod")
B_INVESTMENT= M.addVars(data.producers, data.energy, 
                       vtype= GRB.BINARY, name = "B_data.Invest")
B_INFRALIM = M.addVars(data.producers, data.energy, data.scenario, 
                      vtype = GRB. BINARY, name = "B_InfraLim")
B_EMITLIM = M.addVars(data.producers, data.scenario,  
                      vtype = GRB. BINARY, name = "B_EmitLim")

## Constraints

In [65]:
# Constraints
# ### Follower KKT conditions 

# eq_infraLimit
for pp,ee,ss in itertools.product(data.producers,data.energy,data.scenario):
    eqn = 0
    if ee in data.green:
        eqn = data.CapacityFactor[pp, ee, ss]*(INVESTMENT[pp,ee]+data.InitCapacity[pp, ee]) - PRODUCTION[pp, ee, ss]
    else:
        eqn = INVESTMENT[pp,ee]+data.InitCapacity[pp, ee]- PRODUCTION[pp, ee, ss]
    M.addConstr(eqn >= 0,name = "eq_infraLimit"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_INFRALIM[pp,ee,ss],0,eqn ==0, name = "eq_infraLimit1"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_INFRALIM[pp,ee,ss],1,D_INFRALIMIT[pp,ee,ss] ==0, name = "eq_infraLimit2"+"_".join([pp,ee,ss]))

# eq_emitLimit
for pp,ss in itertools.product(data.producers, data.scenario):
    eqn = data.InitCredits[pp] + FOLL_CARB_BUY[pp] -     quicksum(data.Emission[ee]*PRODUCTION[pp,ee,ss] for ee in data.dirty)
    M.addConstr(eqn >= 0,name = "eq_emitLimit"+"_".join([pp,ss]))
    M.addGenConstrIndicator(B_EMITLIM[pp,ss],0,eqn ==0, name = "eq_emitLimit1"+"_".join([pp,ss]))
    M.addGenConstrIndicator(B_EMITLIM[pp,ss],1,D_EMITLIMIT[pp,ss] ==0, name = "eq_emitLimit2"+"_".join([pp,ss]))

# eq_countryProduction
for ss in data.scenario:
    for cc in data.countries:
        eqn = 0
        for pp in data.producers:
            if (cc,pp) in data.domesticity:
                eqn = eqn + quicksum(PRODUCTION[pp,ee,ss] for ee  in data.energy)
        M.addConstr(eqn == C_PROD[cc,ss], name="eq_countryProduction"+"_".join([cc,ss]))

# eq_production
for pp,ee,ss in itertools.product(data.producers,data.energy,data.scenario):
    eqn = data.probability[ss]*(
        data.LinProdCost[pp,ee] + data.QuadProdCost[pp,ee]*PRODUCTION[pp,ee,ss] -
        quicksum(data.DemInt[cc[0],ss] - data.DemSlope[cc[0],ss]*C_PROD[cc[0],ss] for cc in data.domesticity.select("*",pp)) +
        quicksum(data.DemSlope[cc[0],ss]*PRODUCTION[pp,ee2,ss] for cc in data.domesticity.select("*",pp) for ee2 in data.energy) 
    )+  D_INFRALIMIT[pp,ee,ss] + data.Emission[ee]*D_EMITLIMIT[pp,ss]

    M.addConstr(eqn >= 0,name = "eq_PRODUCTION"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_PRODUCTION[pp,ee,ss],0,eqn ==0, name = "eq_PRODUCTION1"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_PRODUCTION[pp,ee,ss],1,PRODUCTION[pp,ee,ss] ==0, name = "eq_PRODUCTION2"+"_".join([pp,ee,ss]))

# eq_investment
for pp,ee in itertools.product(data.producers,data.green):
    eqn = data.LinInvCost[pp,ee] + data.QuadInvCost[pp,ee]*INVESTMENT[pp,ee] - quicksum(data.CapacityFactor[pp,ee,ss]*D_INFRALIMIT[pp,ee,ss] for ss in data.scenario)
    M.addConstr(eqn >= 0,name = "eq_INVESTMENT"+"_".join([pp,ee]))
    M.addGenConstrIndicator(B_INVESTMENT[pp,ee],0,eqn ==0, name = "eq_INVESTMENT1"+"_".join([pp,ee]))
    M.addGenConstrIndicator(B_INVESTMENT[pp,ee],1,INVESTMENT[pp,ee] ==0, name = "eq_INVESTMENT2"+"_".join([pp,ee]))

# eq_carbonPurchase
for pp in data.producers:
    cc = data.domesticity.select("*", pp)[0][0]
    M.addConstr(CARBON_PRICE[cc] -quicksum(D_EMITLIMIT[pp,ss] for ss in data.scenario) == 0, name = "eq_carbonPurchase"+pp
               )


# ### Leader Constraints 
#     Mininmum consumption constraint
for cc in data.countries:
    M.addConstrs((C_PROD[cc,ss] >= data.minCons[cc] for ss in data.scenario), name = "data.minCons"+cc) 

# Total data.Emission
for ss in data.scenario:
    eqn = {cc:0 for cc in data.countries}
    for (cc,pp) in data.domesticity:
        eqn[cc] = eqn[cc] + quicksum(data.Emission[ee]*PRODUCTION[pp,ee,ss] for ee in data.energy)
    M.addConstrs((TOTAL_EMIT[cc,ss] - eqn[cc]==0 for cc in data.countries), name="Totaldata.Emission"+str(ss))

# Total data.Investment
for ee in data.green:
    eqn = {cc:0 for cc in data.countries}
    for (cc,pp) in data.domesticity:
        eqn[cc] = eqn[cc] + INVESTMENT[pp,ee] 
    M.addConstrs((TOTAL_INV[cc,ee] - eqn[cc]==0 for cc in data.countries), name="TotalInv"+"_".join([cc,ee]))

# Carbon Buy summing
eqn = {cc:0 for cc in data.countries}
for (cc,pp) in data.domesticity:
    eqn[cc] = eqn[cc] + FOLL_CARB_BUY[pp] 
M.addConstrs((LEAD_CARB_BUY[cc] + eqn[cc] == 0 for cc in data.countries), name = "CarbBuySum")

# Carbon Credits >= 0
M.addConstrs((LEAD_CARB_BUY[cc] + data.InitLeaderCredits[cc] + CARB_IMP[cc]  >= 0 for cc in data.countries), 
             name = "CarbCredPos")

# Market clearing goes away
M.addConstr(quicksum(CARB_IMP[cc] for cc in data.countries) >= 0)

M.update()


In [66]:
# ### Leader Objective 
for cc in data.countries:
    M.addConstr(
       1000 * COUNTRY_OBJ[cc] == CARBON_PRICE[cc]*LEAD_CARB_BUY[cc] + # Buy credits from follower
             (data.cciInt + data.cciSlope*quicksum(CARB_IMP[ccdash] for ccdash in data.countries) ) * CARB_IMP[cc] + # Buying carbon credits from international market
               quicksum(data.probability[ss]*TOTAL_EMIT[cc,ss]*data.EmissionValue[cc] for ss in data.scenario)- # self emission
                quicksum(TOTAL_INV[cc,ee]*data.InvestValue[cc,ee] for ee in data.green) +  # self investment
                quicksum(data.probability[ss]*TOTAL_EMIT[cc,ss]*TOTAL_EMIT[c2c,ss]*data.EmissionCrossValue[cc]
                         for c2c in data.countries for ss in data.scenario
                        )- # Emission cross
               quicksum(TOTAL_INV[cc,ee]*TOTAL_INV[c2c,ee]*data.InvestCrossValue[cc,ee] 
                        for ee in data.green for c2c in data.countries), # cross investment    ,
        name = "eq_countryObj"+cc)

activeCountry = {cc:1 for cc in data.countries}
M.setObjective(quicksum(activeCountry[cc]*COUNTRY_OBJ[cc] for cc in data.countries))

In [67]:
M.Params.NonConvex=2
M.Params.Presolve = 2
M.Params.CutPasses = 1
M.Params.PreQLinearize = 1
M.update()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter CutPasses to 1
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter PreQLinearize to 1
   Prev: -1  Min: -1  Max: 2  Default: -1


In [68]:
M.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 105 rows, 204 columns and 416 nonzeros
Model fingerprint: 0x1b7e9075
Model has 2 quadratic constraints
Model has 160 general constraints
Variable types: 116 continuous, 88 integer (88 binary)
Coefficient statistics:
  Matrix range     [1e-01, 4e+00]
  QMatrix range    [1e+00, 1e+01]
  QLMatrix range   [5e+01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [6e-01, 5e+02]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Presolve added 74 rows and 36 columns
Presolve time: 0.01s
Presolved: 239 rows, 258 columns, 767 nonzeros
Presolved model has 58 SOS constraint(s)
Presolved model has 17 bilinear constraint(s)
Variable types: 176 continuous, 82 integer (82 binary)
Presolve removed 133 rows and 134 columns
Presolved: 106 rows, 124 columns, 444 nonzeros

Extra 70 simplex iterations after uncrush

Root relaxation: objective -2.000000e+04, 

In [73]:
CARB_IMP

{'c1': <gurobi.Var CARB_IMP[c1] (value -240.235580718085)>, 'c2': <gurobi.Var CARB_IMP[c2] (value 240.23558072265416)>}

In [70]:
LEAD_CARB_BUY

{'c1': <gurobi.Var LEAD_CARB_BUY[c1] (value 250.0)>, 'c2': <gurobi.Var LEAD_CARB_BUY[c2] (value 175.0)>}

In [71]:
CARBON_PRICE

{'c1': <gurobi.Var CARB_PRI[c1] (value 742.0611533029726)>, 'c2': <gurobi.Var CARB_PRI[c2] (value 748.7108638162026)>}

In [72]:
for kk in INVESTMENT:
    print(kk, "\t\t", INVESTMENT[kk].X)

('p1_1', 'coal') 		 0.0
('p1_1', 'gas') 		 0.0
('p1_1', 'wind') 		 196.3602521607151
('p1_1', 'solar') 		 57.70476036424583
('p1_2', 'coal') 		 0.0
('p1_2', 'gas') 		 0.0
('p1_2', 'wind') 		 196.36025216071513
('p1_2', 'solar') 		 57.70476036424584
('p2_1', 'coal') 		 0.0
('p2_1', 'gas') 		 0.0
('p2_1', 'wind') 		 195.67960852324455
('p2_1', 'solar') 		 66.14678542545784
('p2_2', 'coal') 		 0.0
('p2_2', 'gas') 		 0.0
('p2_2', 'wind') 		 195.67960852324447
('p2_2', 'solar') 		 66.14678542545785
